In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, ALL, State, MATCH
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import boto3
import itemSpecificCleanUp as iscu
import numpy as np
import seaborn as sns
import appGraphs as ag
import ast


s3 = boto3.client('s3')
s3.list_objects(Bucket = 'ebayfindingdata', Prefix = 'shopping/')

keys = [o['Key'] for o in s3.list_objects(Bucket = 'ebayfindingdata', Prefix = 'shopping')['Contents'] if '.csv' in o['Key']]
desc_keys = [o['Key'] for o in s3.list_objects(Bucket = 'ebayfindingdata', Prefix = 'description')['Contents'] if '.csv' in o['Key']]

df = pd.concat([pd.read_csv(s3.get_object(Bucket = 'ebayfindingdata', Key = k)['Body']) for k in keys])
desc_df = pd.concat([pd.read_csv(s3.get_object(Bucket = 'ebayfindingdata', Key = k)['Body']) for k in desc_keys])

df = df.join(desc_df.set_index('ItemID'), on = 'ItemID')

df = df[~df['ItemID'].duplicated()]

df.loc[~df['ItemSpecifics-Type'].isna(), 'ItemSpecifics-Type'] = df[~df['ItemSpecifics-Type'].isna()]['ItemSpecifics-Type'].apply(iscu.cleanUpType)
df.loc[~df['ItemSpecifics-Brand'].isna(),'ItemSpecifics-Brand'] = df[~df['ItemSpecifics-Brand'].isna()]['ItemSpecifics-Brand'].apply(iscu.cleanUpBrand)
df.loc[~df['ItemSpecifics-Skill Level'].isna(), 'ItemSpecifics-Skill Level'] = df[~df['ItemSpecifics-Skill Level'].isna()]['ItemSpecifics-Skill Level'].apply(iscu.cleanUpSkill)
df['Condition'] = df['ConditionID'].apply(iscu.cleanUpCondition)


df.loc[df['ItemSpecifics-Type'].isna(), 'ItemSpecifics-Type'] = df[df['ItemSpecifics-Type'].isna()]['Title'].apply(iscu.extractTypeFromTitle)
df.loc[df['ItemSpecifics-Brand'].isna(),'ItemSpecifics-Brand'] = df[df['ItemSpecifics-Brand'].isna()]['Title'].apply(iscu.extractBrandFromTitle)

df['Model'] = np.nan
df.loc[df['ItemSpecifics-Brand']=='Selmer', 'Model'] = df.loc[df['ItemSpecifics-Brand']=='Selmer', 'Title'].apply(iscu.selmerModel)
df.loc[df['ItemSpecifics-Brand']=='Yamaha', 'Model'] = df.loc[df['ItemSpecifics-Brand']=='Yamaha', 'Title'].apply(iscu.yamahaModel)
df.loc[df['ItemSpecifics-Brand']=='Yanagisawa', 'Model'] = df.loc[df['ItemSpecifics-Brand']=='Yanagisawa', 'Title'].apply(iscu.yanagisawaModel)
df.loc[~df['Model'].isna(), 'ItemSpecifics-Skill Level'] = iscu.extractSkillFromModel(df.loc[~df['Model'].isna()])


In [2]:
df.head()

,ItemID,EndTime,ViewItemURLForNaturalSearch,ListingType,Location,GalleryURL,PictureURL,PrimaryCategoryID,PrimaryCategoryName,BidCount,...,ItemSpecifics-NameValueList-Value,DiscountPriceInfo-OriginalRetailPrice-_currencyID,DiscountPriceInfo-OriginalRetailPrice-value,DiscountPriceInfo-PricingTreatment,DiscountPriceInfo-SoldOneBay,DiscountPriceInfo-SoldOffeBay,Quantity,Description,Condition,Model
0,303116830074,2020-08-04T12:56:22.000Z,https://www.ebay.com/itm/Jupiter-JAS-769GN-sax...,FixedPriceItem,"Saint Louis, Missouri",https://thumbs3.ebaystatic.com/pict/3031168300...,['https://i.ebayimg.com/00/s/MTM4MlgxMjk3/z/HU...,16231,Musical Instruments & Gear:Wind & Woodwind:Ban...,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,very nice Jupiter Jas-769GN Saxophone Eb alto ...,Used,NaN
1,303635927434,2020-08-04T12:57:58.000Z,https://www.ebay.com/itm/Vintage-Conn-Baritone...,Chinese,"Deep River, Connecticut",https://thumbs3.ebaystatic.com/pict/3036359274...,['https://i.ebayimg.com/00/s/MTA2NlgxNjAw/z/gl...,16231,Musical Instruments & Gear:Wind & Woodwind:Ban...,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Fresh from a local estate, you are bidding on ...",Used,NaN
2,254668152053,2020-08-04T13:00:31.000Z,https://www.ebay.com/itm/LAMONTE-TENOR-SAXOPHO...,Chinese,"Trenton, New Jersey",https://thumbs2.ebaystatic.com/pict/2546681520...,['https://i.ebayimg.com/00/s/MTYwMFgxMTk5/z/Yd...,16231,Musical Instruments & Gear:Wind & Woodwind:Ban...,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This is a Lamonte tenor sax. Made in Italy. Th...,Used,NaN
3,233663229380,2020-08-04T13:12:08.000Z,https://www.ebay.com/itm/Used-saxophon-alto-Ya...,Chinese,Bern,https://thumbs1.ebaystatic.com/pict/2336632293...,['https://i.ebayimg.com/00/s/NzE5WDEwMjQ=/z/rj...,16231,Musical Instruments & Gear:Wind & Woodwind:Ban...,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Present to you the Used saxophon alto Yamaha Y...,Used,YAS25
4,233663229383,2020-08-04T13:12:08.000Z,https://www.ebay.com/itm/Used-saxophon-alto-Ya...,Chinese,Bern,https://thumbs4.ebaystatic.com/pict/2336632293...,['https://i.ebayimg.com/00/s/NzE4WDEwMjQ=/z/WA...,16231,Musical Instruments & Gear:Wind & Woodwind:Ban...,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Present to you the Used saxophon alto Yamaha Y...,Used,YAS62
